Only run this notebook if you have verified your ChromeDriver setup in the `test_*.ipynb` notebooks.

We are scraping everlane, which has implemented an infinite scroll system within their ecommerce site.

Infinite scroll means there is no pagination number within the page, and it will scroll as long as there is a next page. Because you can't see the end from where you look, you will have to apply a `while` loop to get it done.

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException
from splinter.exceptions import ElementDoesNotExist

In [2]:
# Open a Chrome window using Splinter
browser = Browser('chrome')

The chromedriver version (126.0.6478.126) detected in PATH at C:\Users\jon_1\bin\chromedriver.exe might not be compatible with the detected chrome version (127.0.6533.90); currently, chromedriver 127.0.6533.99 is recommended for chrome 127.*, so it is advised to delete the driver in PATH and retry


In [3]:
# Setup the URL you want to scrape
url = "https://www.everlane.com/products/womens-organic-cotton-crew-sweater-canvas-black?collection=womens-best-sellers"

In [4]:
# Use the browser to visit the URL
browser.visit(url)

In [5]:
# Close popup
try:
    browser.find_by_text('No Thanks ').first.click()
except IndexError:
    pass
except ElementDoesNotExist:
    pass

In [6]:
# Gather the HTML from the current page in the browser
html = browser.html

In [7]:
html

'<html lang="en" class=""><head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1.0"><link rel="icon" href="https://media.everlane.com/image/upload/c_scale/c_fill,dpr_auto,f_auto,q_auto,w_auto/v1/static/favicon4.ico" type="image/x-icon"><link rel="apple-touch-icon apple-touch-icon-precomposed" sizes="76x76" href="https://media.everlane.com/image/upload/c_fill,dpr_1,f_auto,g_face:center,h_76,q_auto,w_76/v1/static/1x_ipad.png"><link rel="apple-touch-icon apple-touch-icon-precomposed" sizes="167x167" href="https://media.everlane.com/image/upload/c_fill,dpr_1,f_auto,g_face:center,h_167,q_auto,w_167/v1/static/2x_ipad_pro.png"><link rel="apple-touch-icon apple-touch-icon-precomposed" sizes="152x152" href="https://media.everlane.com/image/upload/c_fill,dpr_1,f_auto,g_face:center,h_152,q_auto,w_152/v1/static/2x_ipad.png"><link rel="apple-touch-icon apple-touch-icon-precomposed" sizes="120x120" href="https://media.everlane.com/image/upload/c_fill,dpr_1,f_a

In [8]:
# Use Beautiful Soup to parse the HTML tags into objects.
soup = BeautifulSoup(html, 'html.parser')

In [9]:
rating_value_list = soup.find_all('meta', attrs={"itemprop": "ratingValue"})
rating_value_list

[<meta content="5" itemprop="ratingValue"/>,
 <meta content="5" itemprop="ratingValue"/>,
 <meta content="5" itemprop="ratingValue"/>,
 <meta content="5" itemprop="ratingValue"/>,
 <meta content="5" itemprop="ratingValue"/>,
 <meta content="5" itemprop="ratingValue"/>,
 <meta content="5" itemprop="ratingValue"/>,
 <meta content="5" itemprop="ratingValue"/>]

In [10]:
comments_list = soup.find_all('div', attrs={"class", "bv-content-summary-body-text"})
comments_list

[<div class="bv-content-summary-body-text"> <p>I’m going to be reaching for this sweater year-round. The quality is great and the slightly roomy oversized look is so easy to wear! I got it in navy and will order more colours. Love it!!</p> </div>,
 <div class="bv-content-summary-body-text"> <p>I have five of these sweaters; in fact this is a replacement for one I actually wore out. I like their loose, boxy style but also the weight and quality of the cotton knit. I like cotton weather, particularly in an ever warming world. If anything, Everlane should produce more colors.</p> </div>,
 <div class="bv-content-summary-body-text"> <p>This sweater is beautiful. It drapes so well and is super comfy. The rib details along the neckline and sleeve cuffs are perfection. Will definitely be adding more colors to my knitwear collection. 10/10!</p> </div>,
 <div class="bv-content-summary-body-text"> <p>This is the second sweater I bought in this style - purchased a small and medium. Small is a bett

#### Pagination

In [11]:
# Trigger link by finding the exact css to click on
browser.find_by_css('.bv-content-btn.bv-content-btn-pages.bv-content-btn-pages-load-more.bv-focusable').first.click()

In [12]:
# As we don't know when does the infinite scroll end, we need a while loop
import time
t = 2 # 2 second delay

page_num = 1
more_bool = True

while more_bool:
    try:
        # Gather the HTML from the current page in the browser
        html = browser.execute_script("return document.documentElement.outerHTML;")
        
        # Use Beautiful Soup to parse the HTML tags into objects.
        soup = BeautifulSoup(html, 'html.parser')
        
        # This is where you put in the logic to scrap the reviews section
        # You should tailor this portion for you own use case.
        print(f"--- Page {page_num} --")
        comments_list = soup.find_all('div', attrs={"class", "bv-content-summary-body-text"})
        print(comments_list)
        print("Number of comments:", len(comments_list))

        # The below line returns a list of tag objects
        browser.find_by_css('.bv-content-btn.bv-content-btn-pages.bv-content-btn-pages-load-more.bv-focusable').first.click()
        page_num += 1
        time.sleep(t)
    except StaleElementReferenceException as e:
        print(e)
        more_bool = False
        pass
    except IndexError as e:
        print(e)
        more_bool = False
        pass
    except ElementDoesNotExist as e:
        print(e)
        more_bool = False
        pass


--- Page 1 --
[<div class="bv-content-summary-body-text"> <p>I’m going to be reaching for this sweater year-round. The quality is great and the slightly roomy oversized look is so easy to wear! I got it in navy and will order more colours. Love it!!</p> </div>, <div class="bv-content-summary-body-text"> <p>I have five of these sweaters; in fact this is a replacement for one I actually wore out. I like their loose, boxy style but also the weight and quality of the cotton knit. I like cotton weather, particularly in an ever warming world. If anything, Everlane should produce more colors.</p> </div>, <div class="bv-content-summary-body-text"> <p>This sweater is beautiful. It drapes so well and is super comfy. The rib details along the neckline and sleeve cuffs are perfection. Will definitely be adding more colors to my knitwear collection. 10/10!</p> </div>, <div class="bv-content-summary-body-text"> <p>This is the second sweater I bought in this style - purchased a small and medium. Smal

In [13]:
browser.quit()